# Wave (.wav) to Zero Crossing.

This is an attempt to produce synthetic ZC (Zero Crossing) from FS (Full Scan) files. All parts are calculated in the time domain to mimic true ZC, FFT is not used.

Current status: Seems to work well for "easy files", but not for mixed and low amplitude recordings. I don't know why...

Sources in information/inspiration:

- http://users.lmi.net/corben/fileform.htm#Anabat%20File%20Formats
- https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python
- https://github.com/riggsd/zcant/blob/master/zcant/conversion.py


In [1]:
import numpy as np
import scipy.io.wavfile as wf
import scipy.signal
import sounddevice
from bokeh.io import output_notebook
from bokeh.plotting import output_file, show, figure
from bokeh.charts import Scatter
from bokeh.models import Range1d, HoverTool

In [2]:
# Load BokehJS.
output_notebook()
# Open plot in new tab.
output_file("zero_crossing.html")

Loading BokehJS ...

In [3]:
# Settings.
#sound_file = 'in_data/Ppip_TE384.wav'
sound_file = 'in_data/Mdau_TE384.wav'
#sound_file = 'in_data/myotis_plecotus_eptesicus_TE384.wav'

cutoff_freq_hz = 18000
zc_divratio = 2

In [4]:
# Debug settings.
play_sound = False
debug = False

In [5]:
# Read the sound file.
(sampling_freq, signal_int16) = wf.read(sound_file, 'rb')
print('Sampling freq in file: ' + str(sampling_freq) + ' Hz.')
print(str(len(signal_int16)) + ' samples.')
if play_sound:
    sounddevice.play(signal_int16, sampling_freq)
    sounddevice.wait()

Sampling freq in file: 38400 Hz.
398012 samples.


In [6]:
# Check if TE, Time Expansion.
if '_TE' in sound_file:
    sampling_freq *= 10
print('Sampling freq: ' + str(sampling_freq) + ' Hz.')

Sampling freq: 384000 Hz.


In [7]:
# Signed int16 to [-1.0, 1.0].
signal = np.array(signal_int16) / 32768 

In [8]:
# Noise level. RMS, root-mean-square.
noise_level = np.sqrt(np.mean(np.square(signal)))
print(noise_level)

0.0303963383506


In [9]:
# Filter. Butterworth.
nyquist = 0.5 * sampling_freq
low = cutoff_freq_hz / nyquist
filter_order = 9
b, a = scipy.signal.butter(filter_order, [low], btype='highpass')
#signal= scipy.signal.lfilter(b, a, signal)
signal= scipy.signal.filtfilt(b, a, signal)

In [10]:
# Add hysteresis around zero to remove noise.
signal[(signal < noise_level) & (signal > -noise_level)] = 0.0

In [11]:
# Check where zero crossings may occur.
sign_diff_array = np.diff(np.sign(signal))

In [12]:
# Extract positive zero passings and interpolate where it occurs.
index_array = []
old_index = None
for index, value in enumerate(sign_diff_array):
    if value in [2., 1., 0.]:
        # Check for raising signal level.
        if value == 2.:
            # From negative directly to positive. Calculate interpolated index.
            x_adjust = signal[index] / (signal[index] - signal[index+1])
            index_array.append(index + x_adjust)
            old_index = None
        elif (value == 1.) and (old_index is None):
            # From negative to zero.
            old_index = index
        elif (value == 1.) and (old_index is not None):
            # From zero to positive. Calculate interpolated index.
            x_adjust = signal[old_index] / (signal[old_index] - signal[index+1])
            index_array.append(old_index + x_adjust)
            old_index = None      
    else:
        # Falling signal level.
        old_index = None

print(len(index_array))
if debug:
    print(index_array[:100])

2045


In [13]:
zero_crossings = index_array[::zc_divratio]
print(len(zero_crossings))

1023


In [14]:
freqs = []
times = []
for index, zero_crossing in enumerate(zero_crossings[0:-1]):
    freq = zero_crossings[index+1] - zero_crossings[index]
    freq_hz = sampling_freq * zc_divratio / freq
    if freq_hz >= cutoff_freq_hz:
        freqs.append(freq_hz)
        times.append(zero_crossing)
print(len(freqs))

985


In [15]:
# Prepare arrays for plotting.
freq_array_khz = np.array(freqs) / 1000.0
time_array_s = np.array(times) / sampling_freq
if debug:
    print(len(freq_array_khz))
    print(freq_array_khz[:100])
    print(time_array_s[:100])

In [17]:
# Plotting using Bokeh.
TOOLS="hover,wheel_zoom,box_zoom,crosshair,undo,redo,reset,resize,pan,previewsave"
# TOOLS="undo,redo,resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,previewsave,box_select,poly_select,lasso_select"
p = figure(tools=TOOLS, toolbar_location="above")
p.title.text = sound_file
p.plot_width = 900 
p.plot_height = 600
p.circle(time_array_s, freq_array_khz, size=2, color="navy", alpha=0.5)

x_len = len(signal_int16) / sampling_freq
y_max = sampling_freq / 1000 / 2

p.xaxis.axis_label="Time (s)"
p.yaxis.axis_label="ZC frequency (kHz)"
p.x_range = Range1d(0, x_len, bounds=(0, x_len))
p.y_range = Range1d(0, y_max, bounds=(0, y_max))

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
         ("Time (s)", "@x"),
         ("Frequency (kHz)", "@y"),
    ]

print('Number of plotted values: ' + str(len(zero_crossings)) + ', divratio: ' + str(zc_divratio))

show(p)

Number of plotted values: 1023, divratio: 2
